# Function to determine the shortest path to fulfill an order

## LP model to determine the shortest path
To determine the shortest path to fulfill an order, we can implement a linear programming model which takes the order items that need to be obtained, and determines the shortest path to collect them all. The model assumes starting from the packaging area, collecting all items, and returning to the packaging area. The model is described below. 

#### Sets, parameters, variables
$I$ : The set of items in the order, as well as the packaging section (labelled as $0$). Note, we can call the order items the nodes to hit.

$d_{i j} :$ The distances between each node $i, j \in I$.

$x_{i j} :$ The decision variable, it equals $1$ if you travel from node $i$ to node $j$ and $0$ if not. 

#### Model
Min $$\sum_{i \in I} \sum_{j \in J} x_{i j} d_{i j}$$
Subject to $$\sum_{i \in I} x_{i j} = 1 \quad \forall j \in I \qquad (1)$$
$$\sum_{j \in I} x_{i j} = 1 \quad \forall i \in I \qquad (2)$$
$$x_{i i} = 0 \quad \forall i \in I \qquad (3)$$
$$x_{i j} + x_{j i} \leq 1 \quad \forall i \in I, j \in I \quad (3)$$

Note - constraint $(1)$ ensures that there is one arc into every node, and constraitn $(2)$ ensures that there is one arc out of every node. Constraint $(3)$ ensures that you don't go to nowhere. Constraint $(4)$ ensures that you cannot return back to the node you just came from.

## Implementation of the function
We can define this model as a function which takes in the order, and returns the shortest path. This has been implemented below. Note that we firstly load in all the distances, as well as the required packages.

# NOTE - things to work out:
1. Need to sort out the bug that it splits into $2$ cycles - i.e. goes packaging $\rightarrow 1 \rightarrow 27 \rightarrow$ 0 then $6 \rightarrow 10 \rightarrow 5$ for example! Should start and end at $0$!
1. Need to work out how best to display the outcome! 

In [1]:
# Import packages
import numpy as np
import xpress as xp
import pandas as pd

In [2]:
# Import data file
distances_data = pd.read_excel('DistanceMatrix.xlsx', sheet_name = "DistanceMatrix Meters")

# Drop the first column of indices
d_dat = distances_data.drop(columns = "Index")

# Show a snippet of the data frame
d_dat

,1,2,3,4,5,6,7,8,9,10,...,88,89,90,91,92,93,94,95,96,Packaging
0,0,3,6,9,12,15,18,21,24,27,...,60,63,66,69,72,75,78,81,84,6
1,3,0,3,6,9,12,15,18,21,24,...,63,66,69,72,75,78,81,84,81,9
2,6,3,0,3,6,9,12,15,18,21,...,66,69,72,75,78,81,84,81,78,12
3,9,6,3,0,3,6,9,12,15,18,...,69,72,75,78,81,84,81,78,75,15
4,12,9,6,3,0,3,6,9,12,15,...,72,75,78,81,84,81,78,75,72,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,75,78,81,84,81,78,75,72,69,66,...,15,12,9,6,3,0,3,6,9,69
93,78,81,84,81,78,75,72,69,66,63,...,18,15,12,9,6,3,0,3,6,72
94,81,84,81,78,75,72,69,66,63,60,...,21,18,15,12,9,6,3,0,3,75
95,84,81,78,75,72,69,66,63,60,57,...,24,21,18,15,12,9,6,3,0,78


In [3]:
def shortest_path(order):
    '''
    A function which takes as input an order which is a list of item numbers. 
    
    Returns the shortest path to collect all items, when starting and ending at the packaging station. 
    '''
    # SORTING SETS, PARAMETERS, VARIABLES
    
    # Sort the order into ascending order
    order.sort()
    
    # Drop the 0 values
    order = [i for i in order if i != 0]
    
    order_2 = order.copy()
    
    # Define I to be the order with 0 attached 
    order_2.insert(0, 0)
    I = order_2
    
    # Insert "Packaging into the order list so we can obtain the desired columns 
    order.insert(0,"Packaging")
    
    # Select desired rows and columns
    d_data = distances_data.loc[distances_data["Index"].isin(order)] # Filter rows
    d_data = d_data[order] # Filter columns

    # Rename the packaging column to 0
    d_data = d_data.rename(columns = {"Packaging" : "0"}) 

    # Select and drop the final row
    first_r = d_data.loc[d_data["0"] == 0]
    d_data = d_data.drop(index = [96])

    # Concat the dataframes so final row is first row
    d = pd.concat([first_r, d_data])
    
    # Turn the data into an array for use
    d = d.to_numpy()
    
    # Generate a list of indices for the set
    I_ind = list(range(0, len(I)))    
    
    # Define the x variable
    x = np.array([xp.var(vartype = xp.binary, name = 'x_{0}_{1}'.format(i, j)) for i in I for j in I], 
                 dtype = xp.npvar).reshape(len(I), len(I))
    
    
    
    
    
    # DEFINE THE PROBLEM, DECLARE VARIABLES AND CONSTRAINTS
    
    # Set the problem
    prob = xp.problem(name = "Prob")
    
    # Add the decision variable to the problem
    prob.addVariable(x)

    
    # Add the constraints:
    
    # only one arc into each node
    prob.addConstraint(
        xp.Sum(x[i, j] for i in I_ind) == 1 for j in I_ind
    )

    # only one arc out of each node
    prob.addConstraint(
        xp.Sum(x[i, j] for j in I_ind) == 1 for i in I_ind
    )

    # have to go to a different node
    prob.addConstraint(
        x[i, i] == 0 for i in I_ind
    )

    # can't go back to the node you just came from, unless there is only 1 item to collect
    if len(I_ind) != 2:
        prob.addConstraint(
            x[i, j] + x[j, i] <= 1 for i in I_ind for j in I_ind
        )

        
        
    # DEFINE AND ADD OBJECTIVE
    
    # Define the objective function
    obj = xp.Sum(xp.Sum(x[i, j]*d[i, j] for i in I_ind) for j in I_ind)

    # Set the problems objective function
    prob.setObjective(obj, sense = xp.minimize)
    
    
    # WRITE AND SOLVE PROBLEM
    # Write and solve the problem
    prob.write("problem","lp") # Used to look for cause of infeasibility
    prob.solve()
    
    
    # DEFINE OUTPUTS
    
    # Obtain optimal x values, and objective value
    soln = prob.getSolution(x)
    total_distance = prob.getObjVal()
    
    # Set an empty array for arcs
    arcs = []
    
    # Determine the arcs
    for i in I_ind :
        for j in I_ind :
            if soln[i, j] == 1 :
                arcs.append([I[i], I[j]])

    return soln, total_distance, arcs

In [4]:
# Test 1
soln, tot, arcs = shortest_path([6, 0, 0, 0, 0])
print()
print(f"The solution matrix is:")
print(soln)
print(f"The arcs travelled are {arcs}")
print(f"The total distance travelled is {tot}m")

Using the license file found in your Xpress installation. If you want to use this license and no longer want to see this message, use the following code before using the xpress module:
  xpress.init('/Applications/FICO Xpress/xpressmp/bin/xpauth.xpr')
FICO Xpress v9.2.2, Community, solve started 20:34:03, Mar 10, 2024
Heap usage: 389KB (peak 422KB, 103KB system)
Minimizing MILP Prob using up to 4 threads and up to 8192MB memory, with these control settings:
OUTPUTLOG = 1
Original problem has:
         6 rows            4 cols           10 elements         4 entities
Presolved problem has:
         0 rows            0 cols            0 elements         0 entities
LP relaxation tightened
Presolve finished in 0 seconds
Heap usage: 394KB (peak 422KB, 103KB system)
Will try to keep branch and bound tree memory usage below 7.5GB
Starting concurrent solve with dual (1 thread)

 Concurrent-Solve,   0s
            Dual        
    objective   dual inf
 D  42.000000   .0000000
------- optimal --

In [5]:
# Test 2
soln, tot, arcs = shortest_path([50, 30, 0, 0, 0])
print()
print(f"The solution matrix is:")
print(soln)
print(f"The arcs travelled are {arcs}")
print(f"The total distance travelled is {tot}m")

FICO Xpress v9.2.2, Community, solve started 20:34:03, Mar 10, 2024
Heap usage: 394KB (peak 426KB, 106KB system)
Minimizing MILP Prob using up to 4 threads and up to 8192MB memory, with these control settings:
OUTPUTLOG = 1
Original problem has:
        18 rows            9 cols           36 elements         9 entities
Presolved problem has:
         0 rows            0 cols            0 elements         0 entities
Presolve finished in 0 seconds
Heap usage: 399KB (peak 426KB, 106KB system)
Will try to keep branch and bound tree memory usage below 7.5GB
Starting concurrent solve with dual (1 thread)

 Concurrent-Solve,   0s
            Dual        
    objective   dual inf
 D  66.000000   .0000000
------- optimal --------
Concurrent statistics:
      Dual: 0 simplex iterations, 0.00s
Optimal solution found
 
   Its         Obj Value      S   Ninf  Nneg   Sum Dual Inf  Time
     0         66.000000      D      0     0        .000000     0
Dual solved problem
  0 simplex iterations in 0.0

In [6]:
# Test 3
soln, tot, arcs = shortest_path([49, 18, 76, 0, 0])
print()
print(f"The solution matrix is:")
print(soln)
print(f"The arcs travelled are {arcs}")
print(f"The total distance travelled is {tot}m")

FICO Xpress v9.2.2, Community, solve started 20:34:03, Mar 10, 2024
Heap usage: 397KB (peak 430KB, 109KB system)
Minimizing MILP Prob using up to 4 threads and up to 8192MB memory, with these control settings:
OUTPUTLOG = 1
Original problem has:
        28 rows           16 cols           64 elements        16 entities
Presolved problem has:
        14 rows           12 cols           36 elements        12 entities
Presolve finished in 0 seconds
Heap usage: 431KB (peak 431KB, 109KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  2.00e+00] / [ 1.00e+00,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  Objective      [min,max] : [ 9.00e+00,  7.50e+01] / [ 9.00e+00,  7.50e+01]
Autoscaling applied standard scaling

Symmetric problem: generators: 1, support set: 12
 Number of orbits: 6, largest orbit: 2
 Row orbits: 4, row support: 8
Will try to keep branch and bound tree

In [7]:
# Test 4
soln, tot, arcs = shortest_path([88, 70, 35, 2, 0])
print()
print(f"The solution matrix is:")
print(soln)
print(f"The arcs travelled are {arcs}")
print(f"The total distance travelled is {tot}m")

FICO Xpress v9.2.2, Community, solve started 20:34:03, Mar 10, 2024
Heap usage: 401KB (peak 433KB, 111KB system)
Minimizing MILP Prob using up to 4 threads and up to 8192MB memory, with these control settings:
OUTPUTLOG = 1
Original problem has:
        40 rows           25 cols          100 elements        25 entities
Presolved problem has:
        20 rows           20 cols           60 elements        20 entities
Presolve finished in 0 seconds
Heap usage: 435KB (peak 435KB, 111KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  2.00e+00] / [ 1.00e+00,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  Objective      [min,max] : [ 9.00e+00,  8.10e+01] / [ 9.00e+00,  8.10e+01]
Autoscaling applied standard scaling

Symmetric problem: generators: 1, support set: 20
 Number of orbits: 10, largest orbit: 2
 Row orbits: 5, row support: 10
Will try to keep branch and bound tr

In [8]:
# Test 5
soln, tot, arcs = shortest_path([39, 18, 29, 83, 49])
print()
print(f"The solution matrix is:")
print(soln)
print(f"The arcs travelled are {arcs}")
print(f"The total distance travelled is {tot}m")

FICO Xpress v9.2.2, Community, solve started 20:34:03, Mar 10, 2024
Heap usage: 407KB (peak 439KB, 114KB system)
Minimizing MILP Prob using up to 4 threads and up to 8192MB memory, with these control settings:
OUTPUTLOG = 1
Original problem has:
        54 rows           36 cols          144 elements        36 entities
Presolved problem has:
        27 rows           30 cols           90 elements        30 entities
Presolve finished in 0 seconds
Heap usage: 446KB (peak 469KB, 114KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  2.00e+00] / [ 1.00e+00,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  Objective      [min,max] : [ 9.00e+00,  8.10e+01] / [ 9.00e+00,  8.10e+01]
Autoscaling applied standard scaling

Symmetric problem: generators: 1, support set: 30
 Number of orbits: 15, largest orbit: 2
 Row orbits: 6, row support: 12
Will try to keep branch and bound tr